In [52]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4j_2f_zz
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4j_2f_zz
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [53]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [54]:
# Write your CUDA code to a file
%%writefile hello_cuda.cu
#include <iostream>
__global__ void hello() {
    printf("Hello from CUDA kernel!\n");
}
int main() {
    hello<<<1, 10>>>();
    cudaDeviceSynchronize();
    return 0;
}

Overwriting hello_cuda.cu


In [55]:
# Компилируем код
!nvcc hello_cuda.cu -o hello_cuda

# Запускаем исполняемый файл
!./hello_cuda

Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!


In [56]:
%%writefile kernel.cu
#ifndef __CUDACC__
    #define __CUDACC__
#endif
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <string>
#include <iostream>
#include <iomanip>
#include <time.h>
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>
#define BLOCK_SIZE 512


// функция подсчета суммы элементов вектора на cpu
float sum_calculating_cpu(int* vec, int number_of_elements, int &sum);

// функция подсчета суммы элементов вектора на gpu.
__global__ void sum_calculating_gpu(int* vec, int* result, int number_of_elements);


int main(int argc, char* argv[])
{
	int summ_cpu = 0, summ_gpu = 0;
	srand(time(NULL));
	int number_of_elements = 10000000;
	int to_bites = number_of_elements * sizeof(int);

	// выделение памяти на хосте
	int* vec = (int*)calloc(number_of_elements, sizeof(int));
	int* result_vect = (int*)calloc(number_of_elements, sizeof(int));

	// заполнение вектора рандомными числами
	for (int i = 0; i < number_of_elements; ++i)
	{
		vec[i] = (int)rand() % 100;
	}

	// выделение памяти на гпу
	int* vec_on_device = NULL;
	cudaMalloc((void**)&vec_on_device, to_bites);
	int* result_vec_on_device= NULL;
	cudaMalloc((void**)&result_vec_on_device, to_bites);

	// создание обработчиков событий для отсчета времени
	cudaEvent_t start, stop;
	float time_on_gpu = 0.0f;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	// перенос данных на гпу
	cudaMemcpy(vec_on_device, vec, to_bites, cudaMemcpyHostToDevice);

	// начало события
	cudaEventRecord(start, 0);

	// выполнение подсчета суммы на гпу
	sum_calculating_gpu << < 1, BLOCK_SIZE >> > (vec_on_device, result_vec_on_device, number_of_elements);

	// перенос полученных данных обратно
	cudaMemcpy(result_vect, result_vec_on_device, to_bites, cudaMemcpyDeviceToHost);

	// конец события
	cudaEventRecord(stop, 0);
	// синхронизация
	cudaEventSynchronize(stop);
	// подсчет времени между стартом события и его концом (по факту время вычисления суммы на гпу)
	cudaEventElapsedTime(&time_on_gpu, start, stop);

	// выполнение подсчета суммы на хосте
	float time_on_cpu = sum_calculating_cpu(vec, number_of_elements, summ_cpu);
	summ_gpu = result_vect[0];

	std::cout << "\nВремя подсчета суммы элементов вектора на GPU: " << std::fixed << std::setprecision(2) << time_on_gpu << " млс\n";
	std::cout << "\nВремя подсчета суммы элементов вектора на CPU: " << std::fixed << std::setprecision(2) << time_on_cpu << " млс\n";
	std::cout << "\nСумма CPU: " << summ_cpu;
	std::cout << "\nСумма GPU: " << summ_gpu;

	cudaEventDestroy(start);
	cudaEventDestroy(stop);
	cudaFree(vec_on_device);
	cudaFree(result_vec_on_device);
	free(vec);
	free(result_vect);

	return 0;
}


__global__ void sum_calculating_gpu(int* vec, int* result, int number_of_elements)
{
	// выделение общей памяти
	__shared__ int shared_array[BLOCK_SIZE];
	unsigned int threadId = threadIdx.x;

	// частичные суммы
	int partial = 0;
	for (int i = threadId; i < number_of_elements; i += BLOCK_SIZE)
	{
		partial += vec[i];
	}

	shared_array[threadId] = partial;
	__syncthreads();

	for (unsigned int j = 1; j < blockDim.x; j *= 2)
	{
		if (threadId % (2 * j) == 0)
		{
			shared_array[threadId] += shared_array[threadId + j];
		}
		__syncthreads();
	}

	if (!threadId)
	{
		result[blockIdx.x] += shared_array[0];
	}
}

float sum_calculating_cpu(int* vec, int number_of_elements, int &sum)
{
	clock_t start = clock();

	for (int i = 0; i < number_of_elements; i++)
	{
		sum += vec[i];
	}

	clock_t end = clock();
	float time = ((float)(end - start) / CLOCKS_PER_SEC) * 1000;
	return time;
}

Overwriting kernel.cu


In [57]:
# Компилируем код
!nvcc kernel.cu -o kernel

# Запускаем исполняемый файл
!./kernel


Время подсчета суммы элементов вектора на GPU: 36.76 млс

Время подсчета суммы элементов вектора на CPU: 29.56 млс

Сумма CPU: 495007044
Сумма GPU: 495007044

2 000 - 0.23/0.01

10 000 - 0.26/0.03

50 000 - 0.36/0.13

250 000 - 1.21/0.75

1 000 000 - 3.74/2.83

10 000 000 - 30.71/26.30

In [58]:
!pip install altair

In [59]:
import altair as alt
import pandas as pd

# Данные
data = {
    'Size': [2_000, 10_000, 50_000, 250_000, 1_000_000, 10_000_000],
    'CPU_Time': [0.01, 0.03, 0.13, 0.75, 2.83, 26.30],
    'GPU_Time': [0.23, 0.26, 0.36, 1.21, 3.74, 30.71]
}

# Создание DataFrame
df = pd.DataFrame(data)

# Расчет ускорения
df['Speedup'] = df['CPU_Time'] / df['GPU_Time']

# Построение графика
chart = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('Size:Q', title='Размерность вектора'),
    y=alt.Y('Speedup:Q', title='Ускорение'),
    tooltip=['Speedup:Q']
).properties(
    title='Ускорение работы программы на GPU относительно CPU'
)

# Отображение графика
chart

alt.Chart(...)